## Sistemas de Recomendação

Sistemas de Recomendação podem ser definidos, sucintamente, como o sistema responsável por selecionar itens personalizados com base nos interesses do usuário

Neste projeto, será utilizado uma base de dados pequena em um primeiro momento, para se ter uma melhor compreensão e intuição acerca dos sistemas de recomendação. Em sequência vamos utilizar uma [base de dados do MovieLens](https://grouplens.org/datasets/movielens/100k/)

In [1]:
# Base de dados inicial

avaliacoes = {'Ana': 
		{'Freddy x Jason': 2.5, 
		 'O Ultimato Bourne': 3.5,
		 'Star Trek': 3.0, 
		 'Exterminador do Futuro': 3.5, 
		 'Norbit': 2.5, 
		 'Star Wars': 3.0},
	 
	  'Marcos': 
		{'Freddy x Jason': 3.0, 
		 'O Ultimato Bourne': 3.5, 
		 'Star Trek': 1.5, 
		 'Exterminador do Futuro': 5.0, 
		 'Star Wars': 3.0, 
		 'Norbit': 3.5}, 

	  'Pedro': 
	    {'Freddy x Jason': 2.5, 
		 'O Ultimato Bourne': 3.0,
		 'Exterminador do Futuro': 3.5, 
		 'Star Wars': 4.0},
			 
	  'Claudia': 
		{'O Ultimato Bourne': 3.5, 
		 'Star Trek': 3.0,
		 'Star Wars': 4.5, 
		 'Exterminador do Futuro': 4.0, 
		 'Norbit': 2.5},
				 
	  'Adriano': 
		{'Freddy x Jason': 3.0, 
		 'O Ultimato Bourne': 4.0, 
		 'Star Trek': 2.0, 
		 'Exterminador do Futuro': 3.0, 
		 'Star Wars': 3.0,
		 'Norbit': 2.0}, 

	  'Janaina': 
	     {'Freddy x Jason': 3.0, 
	      'O Ultimato Bourne': 4.0,
	      'Star Wars': 3.0, 
	      'Exterminador do Futuro': 5.0, 
	      'Norbit': 3.5},
			  
	  'Leonardo': 
	    {'O Ultimato Bourne':4.5,
             'Norbit':1.0,
	     'Exterminador do Futuro':4.0}
}


O banco de dados acima é um dicionário onde a chave é o usuário e o dicionário dentro da chave do usuário são os filmes com as respectivas notas

Vamos utilizar a Filtragem Colaborativa. Sendo assim vamos calcular a similaridade dos usuários. Essa similaridade será nossa medida para podemos medir o quão semelhantes esses usuários são, de forma a usar essa informação para realizar indicações de filmes.

Vamos utilizar a distância euclidiana como a base de nosso índice de similaridade. A distância euclidiana é definida por:

$$\sqrt {\sum_{i=1}^{n} (p_{i} - q_{i})^2}$$ 

Iremos comparar os usuários entre si, de acordo com os filmes que assistiram, computando a distância euclidiana das notas dos filmes. Para termos uma nota de similaridade variando de 0 a 1, sendo que quanto maior a nota maior a similaridade utilizaremos a seguinte transformação:

$$\frac{1}{1+\sqrt {\sum_{i=1}^{n} (p_{i} - q_{i})^2}}$$

In [2]:
from math import sqrt

def euclidiana(base, usuario1, usuario2):
    si = {}
    for item in base[usuario1]:
        if item in base[usuario2]:
            si[item] = 1 
            
    if len(si) == 0: 
        return 0
    
    soma = sum([pow(base[usuario1][item] - base[usuario2][item],2) for item in base[usuario1] if item in base[usuario2]])
        
    return 1/(1+sqrt(soma))        

In [3]:
def get_similares(base, usuario):
    similaridade = [(euclidiana(base, usuario, outro),outro) for outro in base if outro != usuario]
    similaridade.sort()
    similaridade.reverse()
    return similaridade[:30] 

In [4]:
get_similares(avaliacoes, 'Ana')

[(0.4721359549995794, 'Pedro'),
 (0.4142135623730951, 'Adriano'),
 (0.38742588672279304, 'Claudia'),
 (0.3483314773547883, 'Leonardo'),
 (0.3405424265831667, 'Janaina'),
 (0.29429805508554946, 'Marcos')]

In [5]:
for user in avaliacoes:
    print(user)
    print(get_similares(avaliacoes, user))

Ana
[(0.4721359549995794, 'Pedro'), (0.4142135623730951, 'Adriano'), (0.38742588672279304, 'Claudia'), (0.3483314773547883, 'Leonardo'), (0.3405424265831667, 'Janaina'), (0.29429805508554946, 'Marcos')]
Marcos
[(0.6666666666666666, 'Janaina'), (0.3405424265831667, 'Pedro'), (0.29429805508554946, 'Ana'), (0.28172904669025317, 'Claudia'), (0.27792629762666365, 'Adriano'), (0.25824569976124334, 'Leonardo')]
Pedro
[(0.5358983848622454, 'Claudia'), (0.4721359549995794, 'Ana'), (0.38742588672279304, 'Leonardo'), (0.38742588672279304, 'Adriano'), (0.3405424265831667, 'Marcos'), (0.32037724101704074, 'Janaina')]
Claudia
[(0.5358983848622454, 'Pedro'), (0.38742588672279304, 'Ana'), (0.3567891723253309, 'Leonardo'), (0.32037724101704074, 'Janaina'), (0.31451985913875646, 'Adriano'), (0.28172904669025317, 'Marcos')]
Adriano
[(0.4142135623730951, 'Ana'), (0.4, 'Leonardo'), (0.38742588672279304, 'Pedro'), (0.31451985913875646, 'Claudia'), (0.2857142857142857, 'Janaina'), (0.27792629762666365, 'Marc

Como podemos ver acima, utilizando a base de testes podemos ver o score de similaridade relativo ao usuário.

Com essa informação já podemos definir a recomendação por similaridade. Iremos usar a similaridade como um peso. 
Digamos que você queira recomendar filmes para o Leonardo.

Nos utilizaremos todos os outros usuários que assistiram filmes que o Leonardo não assistiu. Iremos, para cada outro usuário, multiplicar o score de similaridade pela nota que este outro usuário deu ao filme, somar o resultado dessas multiplicações e dividir pela soma dos scores de similaridade. Podemos definir da seguinte forma:

$$Nota filme = \frac{\sum_{i=1}^{n} similaridade_{i} * nota_{i}}{\sum_{i=1}^{n} similaridade_{i}}$$



In [6]:
def get_recomendacoes(base, usuario):
    totais = {}
    soma_similaridade = {}
    for outro in base:
        if outro == usuario:
            continue
            
        similaridade = euclidiana(base, usuario, outro)
        
        if similaridade <=0:
            continue
            
        for item in base[outro]:
            if item not in base[usuario]:
                totais.setdefault(item, 0)
                totais[item] += base[outro][item] * similaridade
                soma_similaridade.setdefault(item, 0)
                soma_similaridade[item] += similaridade
                
    rankings = [(round(total/soma_similaridade[item],1), item) for item, total in totais.items()]
    rankings.sort()
    rankings.reverse()
    
    return rankings[:30]   

In [7]:
for user in avaliacoes:    
    print(user)
    print(get_recomendacoes(avaliacoes, user))


Ana
[]
Marcos
[]
Pedro
[(2.5, 'Star Trek'), (2.5, 'Norbit')]
Claudia
[(2.7, 'Freddy x Jason')]
Adriano
[]
Janaina
[(2.2, 'Star Trek')]
Leonardo
[(3.5, 'Star Wars'), (2.8, 'Freddy x Jason'), (2.4, 'Star Trek')]


Acima, podemos ver qual a provável pontuação que o usuário daria para os filmes que ele não assistiu ainda. Por exemplo, Leonardo daria 3.5 para Star Wars, quando utilizado a filtragem colaborativa por usuários. Os resultados de Ana, Marcos e Adriano retornaram vazios já que assistiram todos os filmes da base de dados de teste. Na sequência vamos carregar a base de dados do MovieLens e testar nosso sistema de recomendação.

In [8]:
def carrega_MovieLens():
    filmes = {}
    for linha in open('u.item'):
        (id, titulo) = linha.split('|')[:2]
        filmes[id] = titulo
        
    base = {}
    for linha in open('u.data'):
        (id_usuario, id_filme, nota_filme, tempo) = linha.split('\t')
        base.setdefault(id_usuario, {})
        base[id_usuario][filmes[id_filme]] = float(nota_filme)
        
    return base    

In [9]:
base = carrega_MovieLens()

In [10]:
get_similares(base, '212')

[(1.0, '915'),
 (1.0, '866'),
 (1.0, '856'),
 (1.0, '850'),
 (1.0, '849'),
 (1.0, '837'),
 (1.0, '820'),
 (1.0, '818'),
 (1.0, '81'),
 (1.0, '809'),
 (1.0, '808'),
 (1.0, '801'),
 (1.0, '78'),
 (1.0, '770'),
 (1.0, '759'),
 (1.0, '75'),
 (1.0, '722'),
 (1.0, '71'),
 (1.0, '612'),
 (1.0, '61'),
 (1.0, '596'),
 (1.0, '584'),
 (1.0, '583'),
 (1.0, '580'),
 (1.0, '575'),
 (1.0, '572'),
 (1.0, '57'),
 (1.0, '547'),
 (1.0, '544'),
 (1.0, '53')]

Interessante notar que em uma base de dados maior é possível achar outros usuários perfeitamente igual a outros

In [11]:
get_recomendacoes(base, '1')

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, "Someone Else's America (1995)"),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Prefontaine (1997)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, 'Aiqing wansui (1994)'),
 (4.7, 'Pather Panchali (1955)'),
 (4.6, "Some Mother's Son (1996)"),
 (4.6, 'Anna (1996)'),
 (4.5, "Schindler's List (1993)"),
 (4.5, 'Letter From Death Row, A (1998)'),
 (4.5, 'Close Shave, A (1995)'),
 (4.5, 'Casablanca (1942)'),
 (4.4, 'Third Man, The (1949)'),
 (4.4, 'Rear Window (1954)'),
 (4.4, 'Faust (1994)'),
 (4.4, 'Everest (1998)'),
 (4.4, 'Bitter Sugar (Azucar Amargo) (1996)'),
 (4.3, 'World of Apu, The (Apur Sansar) (1959)'),
 (4.3, 'Vertigo (1958)'),
 (4.3, 'To Kill a Mockingbird (1962)'),
 (4.3, 'Stonewall (1995)'),
 (4.3, 'Some Folks Call It a Sling Blade (1993)'),

Na sequência iremos procurar a similaridade entre os filmes e não mais entre os usuários

In [12]:
def inversor(base):

    filmes = {}
    for user in base:
        for film in base[user]:
            if film not in filmes:
                filmes[film] = {}

    for film in filmes:
        new_dict = {}
        for user in base:
            if film in base[user]:
                new_dict[user] = base[user][film]                      
            filmes[film] = new_dict
    return filmes

In [13]:
filmes = inversor(avaliacoes)
filmes

{'Freddy x Jason': {'Ana': 2.5,
  'Marcos': 3.0,
  'Pedro': 2.5,
  'Adriano': 3.0,
  'Janaina': 3.0},
 'O Ultimato Bourne': {'Ana': 3.5,
  'Marcos': 3.5,
  'Pedro': 3.0,
  'Claudia': 3.5,
  'Adriano': 4.0,
  'Janaina': 4.0,
  'Leonardo': 4.5},
 'Star Trek': {'Ana': 3.0, 'Marcos': 1.5, 'Claudia': 3.0, 'Adriano': 2.0},
 'Exterminador do Futuro': {'Ana': 3.5,
  'Marcos': 5.0,
  'Pedro': 3.5,
  'Claudia': 4.0,
  'Adriano': 3.0,
  'Janaina': 5.0,
  'Leonardo': 4.0},
 'Norbit': {'Ana': 2.5,
  'Marcos': 3.5,
  'Claudia': 2.5,
  'Adriano': 2.0,
  'Janaina': 3.5,
  'Leonardo': 1.0},
 'Star Wars': {'Ana': 3.0,
  'Marcos': 3.0,
  'Pedro': 4.0,
  'Claudia': 4.5,
  'Adriano': 3.0,
  'Janaina': 3.0}}

In [14]:
get_similares(filmes,'Star Wars')

[(0.38742588672279304, 'Freddy x Jason'),
 (0.32037724101704074, 'O Ultimato Bourne'),
 (0.2989350844248255, 'Star Trek'),
 (0.29429805508554946, 'Norbit'),
 (0.252650308587072, 'Exterminador do Futuro')]

Utilizando a base de filmes como chaves primárias conseguimos encontrar quais filmes são similares entre si. Continua a mesma lógica do sistema por usuários. Para exemplificar, vamos pegar o Leonardo. Utilizando filtragem baseada em itens, qual nota ele daria para Freedy x Jason, sabendo que ele assistiu O Ultima Bourne, O Exterminador do Futuro e Norbit?

Similarmente a filtragem por usuários, multiplicasse o score de similaridade entre os filmes, pela nota do filme assistido. Soma-se isso para todos os filmes assistidos e dividi-se pela soma das notas de similaridade.

Para realizarmos esses cálculos vamos criar um dicionário com a lista de similaridade de filmes entre si

In [15]:
def calculo_itens_smilares(base_filmes):
    result = {}
    for item in base_filmes:
        notas = get_similares(base_filmes, item)
        result[item] = notas
    return result

In [16]:
itens_similares_teste = calculo_itens_smilares(filmes)
itens_similares_teste

{'Freddy x Jason': [(0.4494897427831781, 'Norbit'),
  (0.38742588672279304, 'Star Wars'),
  (0.3483314773547883, 'Star Trek'),
  (0.3483314773547883, 'O Ultimato Bourne'),
  (0.2402530733520421, 'Exterminador do Futuro')],
 'O Ultimato Bourne': [(0.3483314773547883, 'Freddy x Jason'),
  (0.32037724101704074, 'Star Wars'),
  (0.3090169943749474, 'Exterminador do Futuro'),
  (0.2553967929896867, 'Star Trek'),
  (0.1886378647726465, 'Norbit')],
 'Star Trek': [(0.3483314773547883, 'Freddy x Jason'),
  (0.32037724101704074, 'Norbit'),
  (0.2989350844248255, 'Star Wars'),
  (0.2553967929896867, 'O Ultimato Bourne'),
  (0.20799159651347807, 'Exterminador do Futuro')],
 'Exterminador do Futuro': [(0.3090169943749474, 'O Ultimato Bourne'),
  (0.252650308587072, 'Star Wars'),
  (0.2402530733520421, 'Freddy x Jason'),
  (0.20799159651347807, 'Star Trek'),
  (0.1918253663634734, 'Norbit')],
 'Norbit': [(0.4494897427831781, 'Freddy x Jason'),
  (0.32037724101704074, 'Star Trek'),
  (0.2942980550855

Agora, com esses dados já calculados podemos definir a função que retorna as indicações por usuário, utilizando a similaridade por filmes que ele já viu 

In [17]:
def get_recomendacoes_itens(base_usuario, similaridade_itens, usuario):
    notas_usuario = base_usuario[usuario]
    notas = {}
    total_similaridade = {}
    for item, nota in notas_usuario.items():
        for similaridade, item2 in similaridade_itens[item]:
            if item2 in notas_usuario:
                continue
            notas.setdefault(item2, 0)
            notas[item2] += similaridade*nota
            total_similaridade.setdefault(item2, 0)
            total_similaridade[item2] += similaridade
    rankings = [(round(score/total_similaridade[item],1), item) for item, score in notas.items()]
    rankings.sort()
    rankings.reverse()
    return rankings[:30]
                

In [18]:
get_recomendacoes(avaliacoes, 'Leonardo')

[(3.5, 'Star Wars'), (2.8, 'Freddy x Jason'), (2.4, 'Star Trek')]

In [19]:
get_recomendacoes_itens(avaliacoes, itens_similares_teste, 'Leonardo')

[(3.2, 'Star Wars'), (2.9, 'Star Trek'), (2.9, 'Freddy x Jason')]

Na base de dados de teste podemos ver como isso afeta as notas dadas e as posições. Isso fica ainda mais perceptível na base da dados do MovieLens

In [20]:
base_filmes = inversor(base) 

In [21]:
itens_similares_base = calculo_itens_smilares(base_filmes) 

In [22]:
get_recomendacoes(base, '2')

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, "Someone Else's America (1995)"),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Prefontaine (1997)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, 'Aiqing wansui (1994)'),
 (4.7, 'Pather Panchali (1955)'),
 (4.7, 'Anna (1996)'),
 (4.6, "Some Mother's Son (1996)"),
 (4.6, 'Maya Lin: A Strong Clear Vision (1994)'),
 (4.6, 'Letter From Death Row, A (1998)'),
 (4.6, 'Innocents, The (1961)'),
 (4.6, 'Close Shave, A (1995)'),
 (4.5, 'Shawshank Redemption, The (1994)'),
 (4.5, "Schindler's List (1993)"),
 (4.5, 'Faust (1994)'),
 (4.5, 'Casablanca (1942)'),
 (4.5, 'Bitter Sugar (Azucar Amargo) (1996)'),
 (4.4, 'World of Apu, The (Apur Sansar) (1959)'),
 (4.4, 'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.4, 'Usual Suspects, The (1995)'),
 (

In [23]:
get_recomendacoes_itens(base, itens_similares_base, '2')

[(5.0, 'U.S. Marshalls (1998)'),
 (5.0, 'Thin Blue Line, The (1988)'),
 (5.0, 'Stuart Saves His Family (1995)'),
 (5.0, 'Santa Clause, The (1994)'),
 (5.0, 'Robocop 3 (1993)'),
 (5.0, 'Rhyme & Reason (1997)'),
 (5.0, 'Race the Sun (1996)'),
 (5.0, 'Quest, The (1996)'),
 (5.0, 'Prophecy II, The (1998)'),
 (5.0, 'Pollyanna (1960)'),
 (5.0, 'Night Flier (1997)'),
 (5.0, 'Mark of Zorro, The (1940)'),
 (5.0, 'Man of the House (1995)'),
 (5.0, 'Mamma Roma (1962)'),
 (5.0, 'Major Payne (1994)'),
 (5.0, "Lover's Knot (1996)"),
 (5.0, 'Last Klezmer: Leopold Kozlowski, His Life and Music, The (1995)'),
 (5.0, 'Lady of Burlesque (1943)'),
 (5.0, 'Judgment Night (1993)'),
 (5.0, 'Innocent Sleep, The (1995)'),
 (5.0, 'Incognito (1997)'),
 (5.0, "I Don't Want to Talk About It (De eso no se habla) (1993)"),
 (5.0, 'Glass Shield, The (1994)'),
 (5.0, 'Friday (1995)'),
 (5.0, 'Faust (1994)'),
 (5.0, 'Farewell to Arms, A (1932)'),
 (5.0, 'Far From Home: The Adventures of Yellow Dog (1995)'),
 (5.0, 'Fac

Como pudemos ver, os filmes indicados mudaram completamente! Sendo assim, devemos saber os prós e contras na hora de optar por algum sistema de recomendação. Como computamos anteriormente a similaridade entre os itens precisamos realizar apenas uma busca dos dadas já calculados, diferentemente do que ocorre com a filtragem por usuários. Além disso é bom notar que se tiver um conjunto de dados esparsos, ou seja, onde pessoas deram notas para produtos diferentes, com baixa intersecção o sistema de recomendação por item se mostra mais eficiente.